## Check if there is pig or other contamination:
- Create pangenome sketches at k51: Need a k51 db for this
- Gather these against the euk db
- Create sketches at k31 with just the CDS
- Use these CDS sketches as a db for gather x the metagenomes
- Most euk DNA non-coding.

Exclude E. coli from this? Because getting the CDS from that is very large
For now yes. 

In [ ]:
# try fetchMG (my preference tbh)
/home/amhorst/.local/bin/fetchMGs extraction \
-t 30 /group/ctbrowngrp2/amhorst/2025-pig-corespp/results/functional_profile_corespp/Floccifex_porci/MAGs/AtH2023_SRR5240748_MAG07.fasta \
"genome" ./test_MAG

# run on all mags through a snakemake
# snakefile
srun --account=ctbrowngrp -p med2 -J prokka -t 24:00:00 -c 48 --mem=60gb --pty bash

# snakemake
snakemake --use-conda --resources mem_mb=60000 --rerun-triggers mtime \
-c 48 --rerun-incomplete -k -s snakefiles/single_copy.smk -n





In [ ]:
# symlink gather results
ln -s /group/ctbrowngrp2/amhorst/2025-pigparadigm/results/gatherxgtdb+bins.species .

# create a presence absence table from all gather results
mamba activate pangenomics_dev 

sourmash scripts gather_tables \
gather/*.csv \
-c "intersect_bp" -p \
-o 251104_gatherout.species.pig.csv 

# run branchwater for all the core species in the db
- Update the db to the most current one titus made, to get all the kmers for each species
- Compare the new kmers to kmers gtdb
- Compare % of pangenome kmers found in a metagenome
- Branchwater cli 

## Also do a branchwter search with just the pangenome from GTDB

In [ ]:
# need to make sketches of these first. 
# Both with all MAGs and only GTDB. So that we can compare. 
# snakefile
srun --account=ctbrowngrp -p bmh -J manysearch -t 1:00:00 -c 40 --mem=120gb --pty bash

# snakemake
snakemake --use-conda --resources mem_mb=120000 --rerun-triggers mtime \
-c 40 --rerun-incomplete -k -s snakefiles/single_copy.smk -n

# run manysearch
mamba activate branchwater 
sourmash scripts manysearch pang_sketch.gtdb+mags.txt '/group/ctbrowngrp2/amhorst/2025-pigparadigm/resources/3217_metag.txt' \
-o 251204_manysearch.3216.gtdb+mags.csv -c 40 -t 0 && \
sourmash scripts manysearch pang_sketch.gtdb.txt '/group/ctbrowngrp2/amhorst/2025-pigparadigm/resources/3217_metag.txt' \
-o 251204_manysearch.3216.gtdb.csv -c 40 -t 0

find "$PWD" -type f -name "*.gtdb+mags.k31.rn.sig" > pang_sketch.gtdb+mags.txt
find "$PWD" -type f -name "*.gtdb.k31.rn.pang.sig" > pang_sketch.gtdb.txt


In [ ]:
srun --account=ctbrowngrp -p bmh -J bw -t 6:00:00 -c 1 --mem=10gb --pty bash


branchwater-client --full --sig /group/ctbrowngrp2/amhorst/2025-pig-corespp/results/pangenome/Oscillibacter_segnis/Oscillibacter_segnis.gtdb+mags.k21.sig -o /group/ctbrowngrp2/amhorst/2025-pig-corespp/results/branchwater/Oscillibacter_segnis.gtdb+mags.k21.csv --retry 15

In [ ]:
# get sizes of pangenomes
srun --account=ctbrowngrp -p med2 -J pangenome_sketch -t 1:00:00 -c 10 --mem=20gb --pty bash

mamba activate branchwater
snakemake --use-conda --resources mem_mb=20000 --rerun-triggers mtime \
-c 10 --rerun-incomplete -k -s snakefiles/pangenome_sketch.smk -n

In [ ]:
# do we have at k21? Needed for bw
/home/ctbrown/scratch3/sourmash-midgie-raker/outputs.ath/rename/gtdb+bins.species.sig.zip

gtdb+bins.species.k21.sig.zip

In [ ]:
# do tax on these results ( use tax_metag.smk)
srun --account=ctbrowngrp -p med2 -J tax -t 1:00:00 -c 40 --mem=20gb --pty bash

mamba activate branchwater

snakemake --use-conda --resources mem_mb=50000 --rerun-triggers mtime \
-c 40 --rerun-incomplete -k -s snakefiles/tax_metag.smk -n

# results go into 251105_taxonomy.ipynb 

In [ ]:
# print core microbes
# see 251104_pick_core_spp.ipynb

In [ ]:
grep -l "UBA2868 sp004552595" *.csv | wc -l

In [ ]:
# srun  for notebook 
srun --cpus-per-task=4 --mem=16G --time=04:00:00 --pty bash

srun --account=ctbrowngrp -p med2 \
-J notebook -t 2:00:00 -c 2 --mem=16gb --pty bash

mamba activate jupyter
jupyter notebook --no-browser --port=8888 --ip=0.0.0.0

# copy paste the link into the kernel selecter


In [ ]:
# spp function
srun --account=ctbrowngrp -p med2 -J funprofiler -t 16:00:00 -c 4 --mem=70gb --pty bash

# snakemake
snakemake --use-conda --resources mem_mb=70000 --rerun-triggers mtime \
-c 4 --rerun-incomplete -k -s snakefiles/function.smk -n


In [ ]:
# have a set of samples im pretty sure are not cow but pig gut. I want to gather x the euks
# snakefile

srun --account=ctbrowngrp -p med2 -J gather -t 6:00:00 -c 16 --mem=50gb --pty bash

# snakemake
snakemake --use-conda --resources mem_mb=50000 --rerun-triggers mtime \
-c 16 --rerun-incomplete -k -s snakefiles/test_gather.euk.smk -n

# euk db 
/group/ctbrowngrp/sourmash-db/genbank-euks-2025.01/vertebrates.k51.sig.zip


In [ ]:
srun --account=ctbrowngrp -p med2 -J funprofiler -t 4:00:00 -c 32 --mem=70gb --pty bash


# move files that already have output
for f in *.fasta; do
    if [ -d "../prokka/${f%.fasta}" ]; then
        mv "$f" done/
    fi
done

In [ ]:
# test code
sourmash scripts abundhist \
<metagenome>.sig -I <pangenome.sig> \
--save-figure out.png

sourmash scripts abundhist \
/group/ctbrowngrp/irber/data/wort-data/wort-sra/sigs/SRR5371371.sig \
-I /group/ctbrowngrp2/amhorst/2025-pig-corespp/results/pangenome/Prevotella_sp000434975/Prevotella_sp000434975.gtdb+mags.k31.zip \
--figure SRR5371371.x.Prevotella_sp000434975.png

In [ ]:
# compare to as deeply sequenced pig gut metaG SRR8960211 (49GB )
# Cow one SRR5371371 = 45GB
# and a shallower one ERR3211898  (4.6G)

sourmash scripts abundhist \
/group/ctbrowngrp/irber/data/wort-data/wort-sra/sigs/SRR8960211.sig \
-I /group/ctbrowngrp2/amhorst/2025-pig-corespp/results/pangenome/Cryptobacteroides_sp000434935/Cryptobacteroides_sp000434935.gtdb+mags.k31.zip \
--figure SRR8960211.x.Cryptobacteroides_sp000434935.pig.png --bins 100

sourmash scripts abundhist \
/group/ctbrowngrp/irber/data/wort-data/wort-sra/sigs/SRR5371371.sig \
-I /group/ctbrowngrp2/amhorst/2025-pig-corespp/results/pangenome/Cryptobacteroides_sp000434935/Cryptobacteroides_sp000434935.gtdb+mags.k31.zip \
--figure SRR5371371.x.Cryptobacteroides_sp000434935.png --bins 100

sourmash scripts abundhist \
/group/ctbrowngrp/irber/data/wort-data/wort-sra/sigs/SRR8960211.sig \
-I /group/ctbrowngrp2/amhorst/2025-pig-corespp/results/pangenome/Bariatricus_sp004560705/Bariatricus_sp004560705.gtdb+mags.k31.zip \
--figure SRR8960211.x.Bariatricus_sp004560705.pig.png --bins 100

sourmash scripts abundhist \
/group/ctbrowngrp/irber/data/wort-data/wort-sra/sigs/SRR5371371.sig \
-I /group/ctbrowngrp2/amhorst/2025-pig-corespp/results/pangenome/Bariatricus_sp004560705/Bariatricus_sp004560705.gtdb+mags.k31.zip \
--figure SRR5371371.x.Bariatricus_sp004560705.png --bins 100

sourmash scripts abundhist \
/group/ctbrowngrp/irber/data/wort-data/wort-sra/sigs/SRR8960211.sig \
-I /group/ctbrowngrp2/amhorst/2025-pig-corespp/results/pangenome/Lactobacillus_amylovorus/Lactobacillus_amylovorus.gtdb+mags.k31.zip \
--figure SRR8960211.x.Lactobacillus_amylovorus.pig.png --bins 100

sourmash scripts abundhist \
/group/ctbrowngrp/irber/data/wort-data/wort-sra/sigs/SRR5371371.sig \
-I /group/ctbrowngrp2/amhorst/2025-pig-corespp/results/pangenome/Lactobacillus_amylovorus/Lactobacillus_amylovorus.gtdb+mags.k31.zip \
--figure SRR5371371.x.Lactobacillus_amylovorus.png --bins 100

sourmash scripts abundhist \
/group/ctbrowngrp/irber/data/wort-data/wort-sra/sigs/ERR3211898.sig \
-I /group/ctbrowngrp2/amhorst/2025-pig-corespp/results/pangenome/Lactobacillus_amylovorus/Lactobacillus_amylovorus.gtdb+mags.k31.zip \
--figure ERR3211898.x.Lactobacillus_amylovorus.png --bins 100

In [ ]:
# checkm instead of fetchMG (uses universal mgs)
checkm analyze -h
 checkm analyze --nt -x fasta ./ ./test-checkm

#need marker gene file
export CHECKM_DATA_PATH=/home/amhorst/databases/checkm

# make a marker file with prokaryote set
checkm taxon_set 

# taxon set in /home/amhorst/databases/checkm/Prokaryote.ms
checkm taxon_set life Prokaryote Prokaryote.ms

# works I think?
checkm analyze --nt -x fasta /home/amhorst/databases/checkm/Prokaryote.ms ../single_copy_genes/Bariatricus_sp004560705/MAGs/ ./Bariatricus.test 

srun --account=ctbrowngrp -p med2 -J gather -t 6:00:00 -c 16 --mem=50gb --pty bash

# snakemake
snakemake --use-conda --resources mem_mb=60000 --rerun-triggers mtime \
-c 48 --rerun-incomplete -k -s snakefiles/checkm.smk -n


In [ ]:
# move files that are done bc not reoding 600 MAGs 
for d in /group/ctbrowngrp2/amhorst/2025-pig-corespp/results/prokka/Phocaeicola_vulgatus/*/; do f="/group/ctbrowngrp2/amhorst/2025-pig-corespp/results/single_copy_genes/Phocaeicola_vulgatus/MAGs/done/$(basename "$d").fasta"; [[ -f "$f" ]] && mv "$f" /group/ctbrowngrp2/amhorst/2025-pig-corespp/results/single_copy_genes/Phocaeicola_vulgatus/MAGs/; done



In [ ]:
for d in /group/ctbrowngrp2/amhorst/2025-pig-corespp/results/prokka/Phocaeicola_vulgatus/*/; do f="/group/ctbrowngrp2/amhorst/2025-pig-corespp/results/single_copy_genes/Phocaeicola_vulgatus/MAGs/done/$(basename "$d").fasta"; [[ -f "$f" ]] && echo "$f"; done


count=0; for d in /group/ctbrowngrp2/amhorst/2025-pig-corespp/results/prokka/Phocaeicola_vulgatus/*/; do f="/group/ctbrowngrp2/amhorst/2025-pig-corespp/results/single_copy_genes/Phocaeicola_vulgatus/MAGs/done/$(basename "$d").fasta"; [[ -f "$f" ]] && echo "$f" && ((count++)); done; echo "Total files: $count"



In [ ]:
# well do these individ bc taking for everrrrr
branchwater-client --full --sig {input.sig} -o {output.csv} --retry 25

        Gemmiger_qucibialis
        Phocaeicola_vulgatus
        Roseburia_inulinivorans
    Sodaliphilus_sp004557565


branchwater-client --full --sig \
/group/ctbrowngrp2/amhorst/2025-pig-corespp/results/pangenome/Gemmiger_qucibialis/Gemmiger_qucibialis.gtdb+mags.k21.sig \
-o Gemmiger_qucibialis.gtdb+mags.k21.csv --retry 25